In [26]:
# Build a formula evaluator

In [27]:
import pickle

def load_pickled_object(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [28]:
generic_formula_dictionary = load_pickled_object('generic_formula_dictionary.pkl')
series_dict = load_pickled_object('series_dict.pkl')

In [29]:
series_dict["Sheet1"]

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_1', formulas=[None, None], values=[1, 3], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=3, coordinate='B3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_header_cell_column=3), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_2', formulas=[None, None], values=[2, 4], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=3, row=3, coordinate='C3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)

In [30]:
import pandas as pd
import xlcalculator
import ast
import copy
import xlcalculator.ast_nodes as ast_nodes
import xlcalculator.tokenizer as tokenizer

# Series dictionary for example data
series_dict = {
    "Sheet1|col_1|2|2": pd.Series([1, 2, 3, 4, 5]),
    "Sheet1|col_2|2|3": pd.Series([6, 7, 8, 9, 10])
}

# Function to fetch values from series within the provided range
def get_values_from_series(series_tuple):
    series_ids, indexes, _ = series_tuple
    series_values = series_dict[series_ids[0]]
    start_index, end_index = indexes
    return series_values[start_index:end_index]

# Function to sum values from a series
def SUM(series_tuple):
    return sum(get_values_from_series(series_tuple))

# Function to parse and evaluate a formula string
def evaluate_sum(sum_formula: str):
    tree = ast.parse(sum_formula, mode="eval")
    local_env = {"SUM": SUM}
    compiled = compile(tree, filename="<ast>", mode="eval")
    result = eval(compiled, {"__builtins__": {}}, local_env)
    return result

# Function to adjust index of AST nodes
def adjust_indices(node, index_increment):
    if isinstance(node, ast_nodes.FunctionNode):
        adjust_function_node_indices(node, index_increment)
    elif hasattr(node, 'left') and hasattr(node, 'right'):
        adjust_indices(node.left, index_increment)
        adjust_indices(node.right, index_increment)

# Helper function to adjust indices for function nodes
def adjust_function_node_indices(node, index_increment):
    new_args = []
    for arg in node.args:
        new_args.append(adjust_operand_node(arg, index_increment) if isinstance(arg, ast_nodes.OperandNode) else arg)
    node.args = new_args

# Helper function to adjust indices for operand nodes
def adjust_operand_node(arg, index_increment):
    parts = ast.literal_eval(arg.tvalue)
    series_tuple, indexes, deltas = parts
    updated_indexes = (indexes[0] + index_increment, indexes[1] + index_increment)
    updated_tuple = (series_tuple, updated_indexes, deltas)
    updated_tvalue = f"(({repr(series_tuple[0])},), {updated_indexes}, {deltas})"
    return ast_nodes.OperandNode(tokenizer.f_token(tvalue=updated_tvalue, ttype="operand", tsubtype="text"))

# Function to generate a list of formulas by adjusting AST indices
def generate_formulas_list(formula_ast: xlcalculator.ast_nodes.ASTNode, start_index: int, end_index: int):
    return [generate_single_formula(formula_ast, i) for i in range(start_index, end_index + 1)]

# Helper function to generate a single adjusted formula AST
def generate_single_formula(formula_ast, index_increment):
    new_ast = copy.deepcopy(formula_ast)
    adjust_indices(new_ast, index_increment)
    return new_ast

In [31]:
import xlcalculator.ast_nodes as ast_nodes
import xlcalculator.tokenizer as tokenizer

# Define the formula
formula = '(SUM("((\'Sheet1|col_1|2|2\',), (0, 1), (1, 1))")) + (SUM("((\'Sheet1|col_2|2|3\',), (0, 1), (1, 1))"))'

formula_ast = ast_nodes.OperatorNode(tokenizer.f_token(
    tvalue='+', ttype='operator-infix', tsubtype=''
))

formula_ast.left = ast_nodes.FunctionNode(tokenizer.f_token(
    tvalue="SUM", ttype="function", tsubtype=""
))

formula_ast.left.args = [
    ast_nodes.OperandNode(tokenizer.f_token(
        tvalue="(('Sheet1|col_1|2|2',), (0, 1), (1, 1))", ttype="operand", tsubtype="text"
    ))
]

formula_ast.right = ast_nodes.FunctionNode(tokenizer.f_token(
    tvalue="SUM", ttype="function", tsubtype=""
))

formula_ast.right.args = [
    ast_nodes.OperandNode(tokenizer.f_token(
        tvalue="(('Sheet1|col_2|2|3',), (0, 1), (1, 1))", ttype="operand", tsubtype="text"
    ))
]

formula_list = [item for item in generate_formulas_list(formula_ast, 0 , 3)]
assert [str(item) for item in formula_list] == ['(SUM("((\'Sheet1|col_1|2|2\',), (0, 1), (1, 1))")) + (SUM("((\'Sheet1|col_2|2|3\',), (0, 1), (1, 1))"))',
                                                                              '(SUM("((\'Sheet1|col_1|2|2\',), (1, 2), (1, 1))")) + (SUM("((\'Sheet1|col_2|2|3\',), (1, 2), (1, 1))"))',
                                                                              '(SUM("((\'Sheet1|col_1|2|2\',), (2, 3), (1, 1))")) + (SUM("((\'Sheet1|col_2|2|3\',), (2, 3), (1, 1))"))',
                                                                              '(SUM("((\'Sheet1|col_1|2|2\',), (3, 4), (1, 1))")) + (SUM("((\'Sheet1|col_2|2|3\',), (3, 4), (1, 1))"))']

In [32]:
new_ast = formula_list[0]

In [33]:
new_ast.right.args[0].tvalue

"(('Sheet1|col_2|2|3',), (0, 1), (1, 1))"